<a href="https://colab.research.google.com/github/GoeSoft/My_Python_Learn/blob/main/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80_%D0%BF%D0%BE_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai openai-whisper torch langchain langchain-openai langchain-chroma chromadb tiktoken
!apt install git ffmpeg -y

  Cloning https://github.com/pytube/pytube to /tmp/pip-req-build-uaj4m3vv
  Running command git clone --filter=blob:none --quiet https://github.com/pytube/pytube /tmp/pip-req-build-uaj4m3vv
  Resolved https://github.com/pytube/pytube to commit a32fff39058a6f7e5e59ecd06a7467b71197ce35
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.2 MB/s eta 0:00:00


In [5]:


import yt_dlp

YOUTUBE_URL = "https://www.youtube.com/watch?v=lze725unAwU"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'lecture.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'quiet': True
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

print("✅ Аудио скачано через yt-dlp как lecture.mp3")

✅ Аудио скачано через yt-dlp как lecture.mp3


In [6]:
import whisper
import torch

print("Проверка GPU...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Устройство: {device}")

print("Загрузка модели whisper-large...")
model = whisper.load_model("large", device=device)

print("Транскрибация с переводом на английский (task='translate')...")
result = model.transcribe(
    "lecture.mp3",
    task="translate",      # ← любой язык → английский
    language=None,
    fp16=False if device == "cpu" else True
)

english_text = result["text"]
print("✅ Транскрибация завершена.")
print("Пример (первые 400 символов):")
print(english_text[:400] + "...\n")

Проверка GPU...
Устройство: cuda
Загрузка модели whisper-large...


100%|█████████████████████████████████████| 2.88G/2.88G [00:38<00:00, 80.0MiB/s]


Транскрибация с переводом на английский (task='translate')...
✅ Транскрибация завершена.
Пример (первые 400 символов):
 So, I welcome you to the discipline of right-thinking. Today we have the first meeting with you. And at this first meeting, my task is to explain to you all the right-thinking. We have a little time left. I will now tell you about all the right-thinking in 10-15 minutes. And then we will conduct a small session, a small training, a small training. What is right-thinking? And why do we need it? Yo...



In [7]:
with open("methodical_guide_en.txt", "w", encoding="utf-8") as f:
    f.write(english_text)
print(f"📄 Методичка сохранена ({len(english_text)} символов).")

📄 Методичка сохранена (12683 символов).


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

print("Разбиение текста на чанки...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
texts = text_splitter.split_text(english_text)

print("Создание векторной базы...")
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(texts, embeddings)

print(f"✅ Векторная БД создана из {len(texts)} чанков.")

Разбиение текста на чанки...
Создание векторной базы...
✅ Векторная БД создана из 18 чанков.


In [9]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Промпт: отвечать на русском, используя английский контекст
prompt_template = """
Вы — эксперт по материалам лекции. Отвечайте на вопросы пользователя **строго на русском языке**, используя только информацию из следующего контекста.

Контекст (на английском):
{context}

Вопрос (на русском):
{question}

Ответ (на русском):
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT}
)

def ask_russian(question: str) -> str:
    return qa_chain.invoke({"query": question})["result"]

In [10]:
question = "Какие основные темы были рассмотрены в лекции?"
answer = ask_russian(question)

print("❓ Вопрос:", question)
print("\n✅ Ответ:")
print(answer)

❓ Вопрос: Какие основные темы были рассмотрены в лекции?

✅ Ответ:
Основные темы, рассмотренные в лекции, включают в себя понятие права, корреляцию понятия права и юриспруденции, разделение права на теорию и практику, необходимость изучения теории для становления высококвалифицированным юристом, историческое формирование университетов и их факультетов.
